In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('./titanic/train.csv')
target = pd.read_csv('./titanic/test.csv')
submission = pd.read_csv('./titanic/submission.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
X = train.iloc[:,2:]
Y = train.iloc[:,1]

In [5]:
X.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

In [6]:
X = X.iloc[:,[0,2,3,4,-1]]

In [7]:
X.head()

,Pclass,Sex,Age,SibSp,Embarked
0,3,male,22.0,1,S
1,1,female,38.0,1,C
2,3,female,26.0,0,S
3,1,female,35.0,1,S
4,3,male,35.0,0,S


In [8]:
X.shape, Y.shape

((891, 5), (891,))

In [9]:
grouped_mean =  X.groupby(['Sex','Pclass'])['Age'].transform('mean')
X['Age'] = X['Age'].fillna(grouped_mean)

In [10]:
X.isnull().mean()

Pclass      0.000000
Sex         0.000000
Age         0.000000
SibSp       0.000000
Embarked    0.002245
dtype: float64

In [11]:
X[X['Embarked'].isnull()].index

Int64Index([61, 829], dtype='int64')

In [12]:
idx = X[X['Embarked'].isnull()].index
X.drop(index=idx, inplace=True)

In [13]:
Y.drop(index=idx, inplace=True)

In [14]:
Y.shape

(889,)

In [15]:
X.isnull().mean()

Pclass      0.0
Sex         0.0
Age         0.0
SibSp       0.0
Embarked    0.0
dtype: float64

In [16]:
# 범주형 데이터를 -> 수치형
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    889 non-null    int64  
 1   Sex       889 non-null    object 
 2   Age       889 non-null    float64
 3   SibSp     889 non-null    int64  
 4   Embarked  889 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 41.7+ KB


In [17]:
X['Embarked'].unique(), X['Sex'].unique()

(array(['S', 'C', 'Q'], dtype=object), array(['male', 'female'], dtype=object))

In [18]:
X['Embarked'] = X['Embarked'].map({
    'S':0, 
    'C':1, 
    'Q':2
})
X['Sex'] = X['Sex'].map({
    'male':0, 
    'female':1
})
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    889 non-null    int64  
 1   Sex       889 non-null    int64  
 2   Age       889 non-null    float64
 3   SibSp     889 non-null    int64  
 4   Embarked  889 non-null    int64  
dtypes: float64(1), int64(4)
memory usage: 41.7 KB


In [19]:
# 결정트리

In [20]:
from sklearn.tree import DecisionTreeClassifier

In [21]:
dtc = DecisionTreeClassifier()

In [22]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

In [23]:
kfold = KFold(shuffle=True)
crv =  cross_validate(dtc,X,Y,cv=kfold)


In [24]:
crv['test_score'].mean()

0.7907700120611947

In [25]:
# from sklearn.preprocessing  import StandardScaler

In [26]:
t = target.loc[:, X.columns]

In [27]:
dtc.fit(X,Y)

DecisionTreeClassifier()

In [28]:
t['Embarked'] = t['Embarked'].map({
    'S':0, 
    'C':1, 
    'Q':2
})
t['Sex'] = t['Sex'].map({
    'male':0, 
    'female':1
})

In [29]:
grouped_mean =  t.groupby(['Sex','Pclass'])['Age'].transform('mean')
t['Age'] = t['Age'].fillna(grouped_mean)

In [30]:
dtc.predict(t)

array([0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,

In [31]:
submission['Survived'] = dtc.predict(t)

In [32]:
submission.to_csv('tree_submission.csv',index=False)

In [33]:
from lightgbm import LGBMClassifier
lgb =  LGBMClassifier()

In [34]:
kfold = KFold(shuffle=True)
crv =  cross_validate(lgb,X,Y,cv=kfold)

In [36]:
crv['test_score'].mean()

0.8109883831651115

In [39]:
lgb.fit(X,Y)
submission['Survived'] = lgb.predict(t)
submission.to_csv('tree_submission.csv',index=False)